In [95]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree
from nltk.treetransforms import collapse_unary

import pandas as pd
import os
import re
import copy

def pt(t, d=True):
    '''
    Print tree if d is True
    '''
    if d:
        Tree.fromstring(str(t)).pretty_print()

def bin_print(ts):
    '''
    Print binary tree
    '''
    Tree.fromstring(str(C_Tree.to_binary_right(C_Tree.from_string(ts)))).pretty_print()

def clean_features(t):
    '''
    remove all characters between ## in all the nodes of the tree
    '''
    regex = re.compile(r'##.*?##')
    label = regex.sub('', t.label)
    children = [clean_features(c) for c in t.children]
    return C_Tree(label, children)


# genia
g01 = "(ROOT (NONE The) (NONE 4-carboranyl-substituted) (NONE compounds) (NONE -LRB-) (NONE 7) (NONE , ) (NONE 11) (NONE -RRB-) (NONE showed) (NONE antagonistic) (NONE activity) (NONE but) (NONE no) (NONE agonistic) (NONE activity) (NONE even) (NONE in) (NONE the) (NONE presence) (NONE of) (NONE the) (NONE potent) (NONE synergist) (NONE HX630) (NONE .))"
g02 = "(ROOT (NONE In) (NONE parallel) (NONE ,) (NONE NAC) (NONE was) (NONE shown) (NONE to) (NONE down-regulate) (NONE the) (NONE production) (NONE of) (protein (NONE cytokines)) (NONE by) (cell_type (NONE DC)) (NONE as) (NONE well) (NONE as) (NONE their) (NONE surface) (NONE expression) (NONE of) (protein (NONE HLA-DR)) (NONE ,) (protein (NONE CD86)) (NONE -LRB-) (protein (NONE B7-2)) (NONE -RRB-) (NONE ,) (NONE and) (protein (NONE CD40) (NONE molecules)) (NONE both) (NONE at) (NONE the) (NONE basal) (NONE state) (NONE and) (NONE upon) (NONE LPS) (NONE activation) (NONE .))"
g03 = "(ROOT (NONE The) (RNA (protein (NONE MNDA)) (NONE mRNA)) (NONE level) (NONE in) (cell_type (NONE primary) (NONE granulocytes)) (NONE was) (NONE unaffected) (NONE by) (NONE addition) (NONE of) (protein (NONE interferon) (NONE alpha)) (NONE and) (NONE other) (NONE agents) (NONE including) (protein (NONE interferon) (NONE gamma)) (NONE ,) (protein (NONE endotoxin)) (NONE ,) (protein (NONE poly-LRB-I-RRB-) (NONE poly-LRB-C-RRB-)) (NONE ,) (NONE and) (protein (NONE FMLP)) (NONE .))"
g04 = "(ROOT (NONE The) (NONE expression) (NONE of) (NONE c-fos) (NONE ,) (NONE c-jun) (NONE and) (NONE jun) (NONE B) (NONE proto-oncogenes) (NONE was) (NONE studied) (NONE in) (cell_line (protein (NONE phytohemagglutinin)) (NONE -LRB-) (protein (NONE PHA)) (NONE -RRB-) (NONE activated) (NONE peripheral) (NONE blood) (NONE lymphocytes)) (NONE -LRB-) (cell_type (NONE PBL)) (NONE -RRB-) (NONE from) (NONE young) (NONE and) (NONE aged) (NONE humans) (NONE .))"
g05 = "(ROOT (cell_line (NONE E2) (NONE -stimulated) (NONE ER+) (NONE cells)) (NONE were) (NONE more) (NONE susceptible) (NONE to) (NONE lysis) (NONE by) (cell_type (NONE LAK) (NONE cells)) (NONE than) (NONE corresponding) (cell_line (NONE TAM) (NONE -treated)) (NONE or) (cell_type (NONE control) (NONE cells)) (NONE ,) (NONE while) (NONE treatment) (NONE of) (cell_line (NONE ER-cells)) (NONE with) (NONE either) (NONE E2) (NONE or) (NONE TAM) (NONE alone) (NONE did) (NONE not) (NONE alter) (NONE from) (NONE control) (NONE their) (NONE susceptibility) (NONE to) (NONE this) (NONE immune-mediated) (NONE lysis) (NONE .))"
# ptb
p01 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
p02 = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
p03 = "(S (NP (NP (NNP Seven) (NNP Big) (NNP Board) (NNS stocks)) (: --) (NP (NP (NNP UAL)) (, ,) (NP (NNP AMR)) (, ,) (NP (NNP BankAmerica)) (, ,) (NP (NNP Walt) (NNP Disney)) (, ,) (NP (NNP Capital) (NNP Cities\/ABC)) (, ,) (NP (NNP Philip) (NNP Morris)) (CC and) (NP (NNP Pacific) (NNP Telesis) (NNP Group))) (: --)) (VP (VP (VBD stopped) (S (VP (NN trading)))) (CC and) (VP (ADVP (RB never)) (VBD resumed))) (. .))"
p04 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
p05 = "(FRAG (SBAR (IN As) (IN in) (: :) (`` ``) (SQ (NP (PRP You)) (VP (VBD went) (NP (VBG ballooning))) (. ?) (. ?) (. !) (. !))))"
p06 = "(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))"
p07 = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (. .))"
p08 = "(S (NP (DT The) (JJ finger-pointing)) (VP (VBZ has) (ADVP (RB already)) (VP (VBN begun))) (. .))"
p09 = "(S (`` ``) (NP (DT The) (NN equity) (NN market)) (VP (VBD was) (ADJP (JJ illiquid))) (. .))"
p10 = "(S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitalized) (NNS Classics)) (VP (VBP Take) (NP (DT the) (NNP Stage)) (PP (IN in) (NP (NNP Windy) (NNP City))))) (, ,) ('' '') (NP (NNP Leisure) (CC &) (NNP Arts)) (-RRB- -RRB-)))) (, ,) (NP (NP (NP (DT the) (NN role)) (PP (IN of) (NP (NNP Celimene)))) (, ,) (VP (VBN played) (PP (IN by) (NP (NNP Kim) (NNP Cattrall)))) (, ,)) (VP (VBD was) (VP (ADVP (RB mistakenly)) (VBN attributed) (PP (TO to) (NP (NNP Christina) (NNP Haag))))) (. .))"
# spmrl
s01 = "(VROOT (PN (NE-PNC##lem=Grand|case=*|number=*|gender=*## GRAND) (NE-PNC##lem=Rapids|case=nom|number=sg|gender=neut## RAPIDS)) ($,##lem=--|_## ,) (NP (ADJA-NK##lem=3.|case=nom|number=sg|gender=masc|degree=pos## 3.) (NN-NK##lem=Januar|case=nom|number=sg|gender=masc## Januar)) ($LRB##lem=--|_## -LRB-) (NE##lem=afp|case=nom|number=sg|gender=*## afp) ($LRB##lem=--|_## -RRB-) ($.##lem=--|_## .))"
s02 = "(VROOT (S (PP-MO (APPR-AC##lem=in|_## In) (ART-NK##lem=der|case=dat|number=pl|gender=*## den) (NE-NK##lem=USA|case=dat|number=pl|gender=*## USA)) (VAFIN-HD##lem=haben|number=sg|person=3|tense=pres|mood=ind## hat) (PRF-OA##lem=sich|case=acc|number=sg|person=3## sich) (NP-SB (ART-NK##lem=ein|case=nom|number=sg|gender=neut## ein) (AP-NK (NP-AMS (CARD-NK##lem=neun|_## neun) (NN-NK##lem=Jahr|case=acc|number=pl|gender=neut## Jahre)) (ADJA-HD##lem=alt|case=nom|number=sg|gender=neut|degree=pos## altes)) (NN-NK##lem=Mädchen|case=nom|number=sg|gender=neut## Mädchen)) (CVP-OC (VP-CJ (ADJD-MO##lem=erfolgreich|degree=pos## erfolgreich) (PP-MO (APPR-AC##lem=als|_## als) (NN-NK##lem=Geburtshelferin|case=nom|number=sg|gender=fem## Geburtshelferin)) (VVPP-HD##lem=betätigen|_## betätigt)) (KON-CD##lem=und|_## und) (VP-CJ (VVPP-HD##lem=helfen|_## geholfen)))) ($,##lem=--|_## ,) (VP-OC (NP-OA (PPOSAT-NK##lem=sein|case=acc|number=sg|gender=masc## seinen) (ADJA-NK##lem=klein|case=acc|number=sg|gender=masc|degree=pos## kleinen) (NN-NK##lem=Bruder|case=acc|number=sg|gender=masc## Bruder)) (PP-MO (APPRART-AC##lem=zu|case=dat|number=sg|gender=fem## zur) (NN-NK##lem=Welt|case=dat|number=sg|gender=fem## Welt)) (VZ-HD (PTKZU-PM##lem=zu|_## zu) (VVINF-HD##lem=bringen|_## bringen))) ($.##lem=--|_## .))"
s03 = "(NE##lem=USA|case=nom|number=pl|gender=*## USA)"
s04 = "(VROOT (S (NP-SB (ART-NK##lem=der|case=nom|number=sg|gender=neut## Das) (NN-NK##lem=Ministerium|case=nom|number=sg|gender=neut## Ministerium)) (VAFIN-HD##lem=haben|number=sg|person=3|tense=pres|mood=ind## hat) (NP-OA (AVP-MO (ADV-HD##lem=nur|_## nur) (ADV-MO##lem=noch|_## noch)) (ART-NK##lem=der|case=acc|number=sg|gender=fem## die) (NN-NK##lem=Fachaufsicht|case=acc|number=sg|gender=fem## Fachaufsicht))) ($.##lem=--|_## .))"

## Encoding playground

In [96]:
# binary entity trees
ts = g05
tree = C_Tree.from_string(ts)
tree = tree.collapse_unary()
pt(clean_features(tree))

                                                                                                                                                                     ROOT                                                                                                                                                      
           ___________________________________________________________________________________________________________________________________________________________|_____________________________________________________________________________________________________________________________________________________    
      cell_line              |    |        |       |     |    |        cell_type        |         |            cell_line           |           cell_type        |     |       |      |         |         |     |     |    |    |     |    |    |     |    |      |      |         |         |    |          |          |    |  
  ________|_____________     |    |    

In [97]:
tree = C_Tree.from_string(ts)
tree = clean_features(tree)
pt(tree)


encoder = C_NaiveRelativeEncoding(separator="[_]", unary_joiner="[+]", reverse=False, binary=False, binary_direction="R", binary_marker="[*]")
# encoder = C_NaiveDynamicEncoding(separator="[_]", unary_joiner="[+]", reverse=False, binary=False, binary_direction="R", binary_marker="[*]")
# encoder = C_Tetratag(separator="[_]", unary_joiner="[+]", reverse=False, binary_direction="R", binary_marker="[*]")


lc = encoder.encode(copy.deepcopy(tree))
for w,p,f,l in lc.iterrows():
    if f:
        print(f"{str(w):<20} {str(p):<15} {str(f):<75} {str(l):<15}")
    else:
        print(f"{str(w):<20} {str(p):<15} {str(l):<15}")
dt = encoder.decode(lc)
dt = dt.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
print('\n\n')
pt(dt)

print(str(dt))
print(str(tree))
assert str(dt) == str(tree), "Decoded tree does not match original tree"

                                                                                                                                                                     ROOT                                                                                                                                                 
           ___________________________________________________________________________________________________________________________________________________________|________________________________________________________________________________________________________________________________________________    
      cell_line              |    |        |       |     |    |        cell_type        |         |            cell_line           |           cell_type        |     |       |      |   cell_line  |     |     |    |    |     |    |    |     |    |      |      |         |         |    |          |          |    |  
  ________|_____________     |    |        |       |  